<a href="https://colab.research.google.com/github/jamiewu128/cupoy/blob/master/06_fix_content.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from tqdm import tnrange, tqdm_notebook
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba
import jieba.analyse
#jieba.set_dictionary('dict.txt')
#jieba.analyse.set_stop_words('stop_words.txt')

In [0]:
otherdf=pd.read_csv('All_14class.csv')
dupdf=pd.read_csv('unknow_class.csv')

In [4]:
otherdf.info()
dupdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67242 entries, 0 to 67241
Data columns (total 7 columns):
topic         67242 non-null int64
topic_name    67242 non-null object
title         67242 non-null object
content       67242 non-null object
txtlen        67242 non-null int64
url           67242 non-null object
cbodyurl      67242 non-null object
dtypes: int64(2), object(5)
memory usage: 3.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14617 entries, 0 to 14616
Data columns (total 1 columns):
content    14617 non-null object
dtypes: object(1)
memory usage: 114.3+ KB


In [0]:
content=otherdf['content']
sentences = []
for i in tqdm_notebook(range(len(content))):
    tt= content[i]
    sentences.append(jieba.analyse.extract_tags(tt, topK=100, withWeight=False, allowPOS=()))

Building prefix dict from /project/at073-group42/find_outlier/dict.txt ...
Loading model from cache /tmp/jieba.ud3ef02f503236c2d4fd57dbacf157d4d.cache
Loading model cost 0.827 seconds.
Prefix dict has been built succesfully.


In [0]:
other=pd.DataFrame({'content':sentences,'topic':otherdf['topic'],'idx':otherdf.index.values})
other.to_csv('delete_duplicate_top100.csv',index=False)

In [0]:
content=dupdf['content']
sentences = []
for i in tqdm_notebook(range(len(content))):
    tt= content[i]
    sentences.append(jieba.analyse.extract_tags(tt, topK=100, withWeight=False, allowPOS=()))

Building prefix dict from /project/at073-group42/find_outlier/dict.txt ...
Loading model from cache /tmp/jieba.ud3ef02f503236c2d4fd57dbacf157d4d.cache
Loading model cost 0.557 seconds.
Prefix dict has been built succesfully.


In [0]:
out=pd.DataFrame({'content':sentences})

In [0]:
out.to_csv('unknow_class_top100.csv',index=False)

In [0]:
undf=pd.read_csv('unknow_class_top100.csv', converters={'content': eval})
otherdf=pd.read_csv('delete_duplicate_top100.csv', converters={'content': eval})

In [0]:
undf.info()
otherdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14617 entries, 0 to 14616
Data columns (total 1 columns):
content    14617 non-null object
dtypes: object(1)
memory usage: 114.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67242 entries, 0 to 67241
Data columns (total 3 columns):
content    67242 non-null object
topic      67242 non-null int64
idx        67242 non-null int64
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


In [0]:
topic_dic={'旅遊':1,
       '蘋果專區':2,
       '美食':3,
       'AI與大數據':4,
       '職涯':5,
       '美妝保養':6,
       '理財':7,
       '房地產':8,
       '汽車':9,
       '醫療健康':10,
      '財經':11,
      '區塊鏈技術':12,
          '3C產品':13,
          '無人駕駛':14}

In [0]:
from sklearn.cluster import DBSCAN
outlier_detection = DBSCAN(
eps=0.89, min_samples=15, metric='cosine', algorithm='auto',
                    leaf_size=30, p=None, n_jobs=1)

In [0]:
unknow=undf['content']
label=[]
print('count=',len(unknow))
for d in unknow:
    print('>>>> index= ',len(label))
    y=0
    for key in topic_dic.keys():
        t=topic_dic.get(key)
        sdf=otherdf[otherdf.topic==t]
        #sentences=sdf['content'].values
        sentences=sdf['content']
        sentences=sentences.set_value(len(sentences),d)
        vectorizer = TfidfVectorizer(norm=None) ## do not do normalize
        r=vectorizer.fit_transform([' '.join(x) for x in sentences])
        cluster=outlier_detection.fit_predict(r)
        #print('cluster==0 : ',(cluster==0).sum())
        #print('cluster<0 : ',(cluster<0).sum())
        if(cluster[len(cluster)-1]>=0):
            y=t
            #print('[',t,'] put ',d,' in topic ',key)
        #else:
            #print('[',t,'] skip! (cluster label=',cluster[len(cluster)-1],')')
        
    if(y>0):
        label.append(y)
        print('label found : ',y)
    else:
        label.append(0)
        print('label NOT found')


count= 14617
>>>> index=  0
label found :  2
>>>> index=  1
label found :  6
>>>> index=  2
label found :  2
>>>> index=  3
label found :  3
>>>> index=  4
label found :  3
>>>> index=  5
label found :  9
>>>> index=  6
label found :  9
>>>> index=  7
label found :  9
>>>> index=  8
label found :  2
>>>> index=  9
label found :  2
>>>> index=  10
label found :  2
>>>> index=  11
label found :  2
>>>> index=  12
label found :  2
>>>> index=  13
label found :  2
>>>> index=  14
label found :  2
>>>> index=  15
label found :  2
>>>> index=  16
label found :  2
>>>> index=  17
label found :  2
>>>> index=  18
label found :  2
>>>> index=  19
label found :  2
>>>> index=  20
label found :  2
>>>> index=  21
label found :  2
>>>> index=  22
label found :  2
>>>> index=  23
label found :  2
>>>> index=  24
label found :  2
>>>> index=  25
label found :  9
>>>> index=  26
label found :  2
>>>> index=  27
label found :  5
>>>> index=  28
label found :  6
>>>> index=  29
label found :  6
>>>> in

KeyboardInterrupt: 

In [0]:
#topic_name=[]
#for key in topic_dic.keys():
#    topic_name.append(key)
    
#tn=[]
#for l in label:
#    tn.append(topic_name[l-1])





In [0]:
r=pd.DataFrame({'topic':label,'content':unknow})
r._to_csv('unknow_class_top100_dbscan_label.csv',index=False)
r.info()

In [0]:
#combine 14class and unknow predict
data =pd.read_csv('All_14class.csv')
fixed=pd.read_csv('unknow_class_predict.csv')

In [0]:
d1=pd.DataFrame({'topic':data['topic'],'content':data['content']})
d2=pd.DataFrame({'topic':fixed['topic'],'content':fixed['content']})

In [0]:
g=d2.groupby('topic')
g.size()

topic
1     2660
2     1401
3     1922
4      842
5      985
6      215
7     1652
8      342
9      732
10     879
11    1018
12    1472
13     116
14     381
dtype: int64

In [0]:
d2[d2.topic==2]

,topic,content
59,2,讓家人都能快速學會的 Google 地圖家族旅行應用懶人包分享取得連結FacebookTwi...
86,2,蘋果春季發表會將於明日登場，國外科技部落客早已開始爆料，指出Iphone有可能將推出新顏色。...
2043,2,▲TaxiGo推黃金週優惠。（圖／TaxiGo提供）記者賴文萱／台北報導叫車平台TaxiGo...
3666,2,▲RIMOWA透明行李箱。（圖／RIMOWA粉絲頁）文／麗塔很多專家都建議，為了避免太相似造...
3925,2,▲帶小孩出遊。（示意圖／翻攝pixabay）文／麗塔大家一定有在機上被尖叫哭鬧的小孩吵醒的經...
4583,2,你熱愛在全世界各地到處旅行，想要珍藏所有的回憶嗎 ? 那麼千萬別錯過《 RecordMyTr...
4666,2,（圖片來源／BGR）歷經一年的時間，Google 即將發表旗下智慧型手機 Pixel 2，與...
4667,2,今年 9 月，蘋果推出 3 款新 iPhone 時，也較低調地發表了一款周邊配件「AirPo...
4668,2,去年 12 月正式上市時，曾迎來熱銷熱潮、導致產品到貨時間長達 1 個月的 AirPods ...
4669,2,（圖片來源／Wareable）根據外媒《Slash Gear》的報導指出，Apple Wat...


In [0]:
g=d1.groupby('topic')
g.size()

topic
1     2926
2     7569
3     2580
4     5300
5     7808
6     4665
7     4151
8     3508
9     8429
10    8445
11    5880
12    2936
13    1649
14    1396
dtype: int64

In [0]:
df=d2.append(d1)

In [0]:
df['idx']=df.index.values

In [0]:
df.head()

,topic,content,idx
0,1,今年的櫻花期看來都會提早些報到，有想衝一波賞櫻的朋友，可得稍微留意安排囉！每年都讓人引頸期盼...,0
1,6,愛麗森的變身教室合作信箱：niou0808@gmail.com；影音頻道：https://w...,1
2,1,吉米跟髣髣自從來過岩手縣吃過盛岡冷麵，就一直對盛岡冷麵念念不忘，而發現居然在盛岡手工村還能體...,2
3,3,首屆台北米其林今天公布，共有 20 家餐廳上榜，高雄餐旅大學副校長劉喜臨預估將吸引 10 萬...,3
4,3,一本紅色冊子，讓全球餐飲業競逐，饕客趨之若鶩。它能讓業績暴增，卻也能讓人歇業；它是榮耀桂冠，...,4


In [0]:
df.to_csv('All_14class_append_fixed_data.csv',index=False)